In [7]:
import pandas as pd
df = pd.read_csv('./data/movies_train.csv')
testdf = pd.read_csv('./data/movies_test.csv')

In [8]:
df.corr()

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
time,1.000000,0.264675,0.306727,0.623205,0.114153,0.441452
dir_prev_bfnum,0.264675,1.000000,0.131822,0.323521,0.083818,0.283184
dir_prev_num,0.306727,0.131822,1.000000,0.450706,0.014006,0.259674
num_staff,0.623205,0.323521,0.450706,1.000000,0.077871,0.544265
num_actor,0.114153,0.083818,0.014006,0.077871,1.000000,0.111179
box_off_num,0.441452,0.283184,0.259674,0.544265,0.111179,1.000000


# 전처리

In [9]:
# column 삭제
def delete_col(df) :
    del_list = ['title','director']  # 'num_staff','num_actor','dir_prev_num','dir_prev_bfnum' ,
    for i in del_list :
        del df[i]
    return df


# column 생성
def make_col(dataframe, column_name, mapper) :
    new = []
    for i in dataframe[column_name] :
        if i in mapper.keys() :
            new.append(mapper.get(i))
        else :
            new.append(1)
    dataframe['new_' + str(column_name)] = pd.DataFrame(new)
    del dataframe[column_name]
    return dataframe

In [10]:
# 배급사 mapping
def distributor(df, distributor_mapper) :
    return make_col(df, 'distributor', distributor_mapper )
    
# genre mapping
def genre(df) :
    df = pd.get_dummies(df, columns=['genre'], prefix='genre')
    return df

# release time
def release_time(df) :
    years, months, days = [], [], []
    for line in df['release_time'] :
        a = line.split('-')
        years.append(int(a[0]))
        months.append(int(a[1]))
        days.append(a[2])
        
    new_days = []
    for i in range(len(days)) :
        if int(days[i]) >= 20 :
            new_days.append(3)
        elif int(days[i]) >= 10 :
            new_days.append(2)
        else :
            new_days.append(1)    
            
    df['release_year'] = pd.DataFrame(years)
    df['release_month'] = pd.DataFrame(months)
    df['release_day'] = pd.DataFrame(new_days)

    del df['release_time']
    return df

def screen_rat(df) :
    df = pd.get_dummies(df, columns=['screening_rat'], prefix='genre')
    return df

In [11]:
# 배급사 mapper는 정해둔다
distributor_mapper = df['distributor'].value_counts()
distributor_mapper.value_counts()

mapping = {}
for i in range(len(distributor_mapper)):

    if distributor_mapper[i] >= 40 :
        mapping.update({ distributor_mapper.index[i] : 6 })
        
    elif distributor_mapper[i] >= 30 :
        mapping.update({ distributor_mapper.index[i] : 5 })
        
    elif distributor_mapper[i] >= 20 :
        mapping.update({ distributor_mapper.index[i] : 4 })
        
    elif distributor_mapper[i] >= 10 :
        mapping.update({ distributor_mapper.index[i] : 3 })
        
    elif distributor_mapper[i] >= 2 :
        mapping.update({ distributor_mapper.index[i] : 2 })
        
    else :
        mapping.update({ distributor_mapper.index[i] : 1 })


In [12]:
df = df.fillna(0)
df = delete_col(df)
df = distributor(df, mapping)
df = genre(df)
df = release_time(df)
# df = time(df)
# df = staff(df)
# df = actor(df)
df = screen_rat(df)

df

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,new_distributor,genre_SF,genre_공포,genre_느와르,...,genre_애니메이션,genre_액션,genre_코미디,release_year,release_month,release_day,genre_12세 관람가,genre_15세 관람가,genre_전체 관람가,genre_청소년 관람불가
0,96,0.00,0,91,2,23398,6,0,0,0,...,0,1,0,2012,11,3,0,0,0,1
1,130,1161602.50,2,387,3,7072501,4,0,0,1,...,0,0,0,2015,11,2,0,0,0,1
2,123,220775.25,4,343,4,6959083,4,0,0,0,...,0,1,0,2013,6,1,0,1,0,0
3,101,23894.00,2,20,6,217866,5,0,0,0,...,0,0,1,2012,7,2,0,0,1,0
4,108,1.00,1,251,2,483387,2,0,0,0,...,0,0,1,2010,11,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,111,3833.00,1,510,7,1475091,5,0,0,0,...,0,0,0,2014,8,2,0,0,0,1
596,127,496061.00,1,286,6,1716438,4,0,0,0,...,0,0,0,2013,3,2,0,1,0,0
597,99,0.00,0,123,4,2475,4,0,1,0,...,0,0,0,2010,9,3,0,0,0,1
598,102,0.00,0,431,4,2192525,6,0,0,1,...,0,0,0,2015,5,2,0,1,0,0


In [13]:
testdf = testdf.fillna(0)
testdf = delete_col(testdf)
testdf = distributor(testdf, mapping)
testdf = genre(testdf)
testdf = release_time(testdf)
# testdf = time(testdf)
# testdf = staff(testdf)
# testdf = actor(testdf)
testdf = screen_rat(testdf)

testdf

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,new_distributor,genre_SF,genre_공포,genre_느와르,genre_다큐멘터리,...,genre_애니메이션,genre_액션,genre_코미디,release_year,release_month,release_day,genre_12세 관람가,genre_15세 관람가,genre_전체 관람가,genre_청소년 관람불가
0,125,3.005290e+05,2,304,3,2,0,0,1,0,...,0,0,0,2010,1,1,0,0,0,1
1,113,3.427002e+05,4,275,3,4,0,0,0,0,...,0,0,0,2010,1,2,1,0,0,0
2,115,4.206611e+06,3,419,7,6,0,0,0,0,...,0,0,0,2010,1,3,1,0,0,0
3,116,6.913420e+05,2,408,2,4,0,0,0,0,...,0,1,0,2010,2,1,0,1,0,0
4,110,3.173800e+04,1,380,1,6,0,1,0,0,...,0,0,0,2010,2,2,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,78,2.590000e+03,1,4,4,1,0,0,0,0,...,0,0,0,2015,11,2,0,1,0,0
239,86,0.000000e+00,0,18,2,4,0,0,0,1,...,0,0,0,2015,10,3,1,0,0,0
240,107,0.000000e+00,0,224,4,1,0,0,1,0,...,0,0,0,2015,10,3,0,0,0,1
241,102,5.069900e+04,1,68,7,1,0,0,0,0,...,0,0,0,2015,10,3,0,0,0,1


In [14]:
df.corr().sort_values(by='box_off_num', ascending = False)

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,new_distributor,genre_SF,genre_공포,genre_느와르,...,genre_애니메이션,genre_액션,genre_코미디,release_year,release_month,release_day,genre_12세 관람가,genre_15세 관람가,genre_전체 관람가,genre_청소년 관람불가
box_off_num,0.441452,0.293791,0.259674,0.544265,0.111179,1.000000,0.331999,0.088009,-0.017627,0.184869,...,-0.054872,0.181191,0.082780,-0.002497,0.019104,-0.011023,0.033896,0.210368,-0.133525,-0.135180
num_staff,0.623205,0.369657,0.450706,1.000000,0.077871,0.544265,0.472197,0.041529,0.041874,0.209779,...,-0.122539,0.249187,0.108996,-0.032891,-0.002841,0.008254,0.060474,0.246535,-0.273736,-0.085689
time,1.000000,0.266065,0.306727,0.623205,0.114153,0.441452,0.295490,0.004290,-0.009848,0.174151,...,-0.192651,0.166415,0.075757,-0.062575,-0.023969,0.024999,0.017161,0.239238,-0.259244,-0.055087
new_distributor,0.295490,0.184767,0.260738,0.472197,0.042123,0.331999,1.000000,0.044495,0.032871,0.090262,...,0.030787,0.106743,0.035902,-0.060617,-0.008559,-0.065397,0.003353,0.177246,-0.115598,-0.091553
dir_prev_bfnum,0.266065,1.000000,0.396616,0.369657,0.042491,0.293791,0.184767,0.087071,0.014085,0.050141,...,-0.068436,0.099508,0.112761,0.033592,0.047183,-0.035287,0.033362,0.099122,-0.123691,-0.031258
dir_prev_num,0.306727,0.396616,1.000000,0.450706,0.014006,0.259674,0.260738,0.054248,0.050714,0.077037,...,-0.133530,0.183462,0.057303,0.132621,0.034019,0.030678,0.020939,0.196600,-0.166861,-0.085814
genre_15세 관람가,0.239238,0.099122,0.196600,0.246535,0.065280,0.210368,0.177246,0.039323,0.067178,-0.001531,...,-0.116486,0.109911,0.126227,-0.071860,-0.051320,-0.018937,-0.322418,1.000000,-0.303177,-0.511330
genre_느와르,0.174151,0.050141,0.077037,0.209779,-0.010128,0.184869,0.090262,-0.032304,-0.059554,1.000000,...,-0.041340,-0.048027,-0.067569,0.013021,-0.033931,0.008163,-0.098240,-0.001531,-0.092378,0.149692
genre_액션,0.166415,0.099508,0.183462,0.249187,-0.009006,0.181191,0.106743,-0.032926,-0.060700,-0.048027,...,-0.042136,1.000000,-0.068869,-0.013522,0.031853,0.075406,-0.079095,0.109911,-0.094155,0.024687
num_actor,0.114153,0.042491,0.014006,0.077871,1.000000,0.111179,0.042123,-0.000874,0.019558,-0.010128,...,0.152677,-0.009006,0.131034,-0.098869,-0.015063,0.009580,-0.018293,0.065280,0.017003,-0.063550


In [169]:
df.corr().sort_values(by='box_off_num', ascending = False).index

Index(['box_off_num', 'num_staff', 'time', 'new_distributor', 'dir_prev_bfnum',
       'dir_prev_num', 'genre_15세 관람가', 'genre_느와르', 'genre_액션', 'num_actor',
       'genre_SF', 'genre_코미디', 'genre_12세 관람가', 'release_month',
       'release_year', 'release_day', 'genre_미스터리', 'genre_공포', 'genre_서스펜스',
       'genre_드라마', 'genre_뮤지컬', 'genre_애니메이션', 'genre_멜로/로맨스', 'genre_전체 관람가',
       'genre_청소년 관람불가', 'genre_다큐멘터리'],
      dtype='object')

In [172]:
features = ['num_staff', 'time', 'new_distributor', 'dir_prev_bfnum',
       'dir_prev_num', 'genre_15세 관람가', 'genre_느와르', 'genre_액션']

# 모델링

In [173]:
train_Y = df['box_off_num']
train_X = df.drop(['box_off_num'], axis=1)[features]

test_X = testdf[features]

In [150]:
# 저장함수
def save_file(prediction, number) :
    sub = pd.read_csv('./data/submission.csv')
    sub['box_off_num'] = prediction
    sub.to_csv('./submission/submission' + str(number) + '.csv', index=False)
    return print('done')

## 사이킷런

In [174]:
from sklearn import linear_model

lm = linear_model.LinearRegression()
lm.fit(train_X, train_Y)

prediction = lm.predict(test_X)

In [119]:
save_file(prediction, 1)
# 1744049.2329963981	

done


In [184]:
save_file(prediction, 4)
# 1509223.1428086378	 , 'num_staff','num_actor'를 추가하였을 때

done


In [15]:
save_file(prediction, 6)

done


In [28]:
save_file(prediction, 7)
# 1661234.9589730736	

done


In [39]:
save_file(prediction, 8)
# 1507395.893538275	, 'num_staff','num_actor'를 추가하였을 때 , 상영시간 -> get_dummies

done


In [47]:
save_file(prediction, 9)
# 1507395.893538275	 num_staff','num_actor'를 추가하였을 때 , 

# 위와 결과가 같기 때문에 get_dummies와는 큰 관계가 없다

done


In [ ]:
save_file(prediction, 11)
# 1507395.893538275	, 'num_staff','num_actor'를 추가하였을 때
# 상영시간 -> get_dummies
# times 원래 데이터 사용

# 2651221.073888509	

In [87]:
save_file(prediction, 12)
# 1507395.893538275	, 'num_staff','num_actor', 'dir_prev_num'를 추가하였을 때
# 상영시간 -> get_dummies

# 1506004.9532714835

done


In [144]:
prediction = abs(prediction.astype(int))
# 'num_staff','num_actor', 'dir_prev_num'를 추가하였을 때
# release_year 제거
# abs( int )

save_file(prediction, 13)
# 1504505.219927887	

done


In [21]:
save_file(prediction, 16)
# 1507654.7290159652	

done


In [30]:
save_file(prediction, 17)
# 1505315.8479013287	

done


In [43]:
save_file(prediction, 18)
# 	1488993.1435613963	

done


In [152]:
save_file(prediction, 20)
# 	1569824.4586113354	

done


In [168]:
save_file(prediction, 21)
# 0으로 바꿈
# 	2510468.8149881857	

done


In [175]:
save_file(prediction, 22)
# 2323819.3745796373	

done


## tensorflow

In [50]:
import tensorflow as tf
import numpy as np

X = tf.placeholder(shape = [None,25], dtype=tf.float32)
Y = tf.placeholder(shape = [None,1], dtype=tf.float32)

W = tf.Variable(tf.random_normal([25,1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name = 'bias')

H = tf.matmul(X, W) + b
cost = tf.reduce_mean(tf.square(H-Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.0000000002)
train = optimizer.minimize(cost)


sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(5000) :
    _, cost_val = sess.run([train, cost],
                          feed_dict = {
                              X : train_X.values,
                              Y : train_Y.values.reshape(-1,1)
                          })
    if step % 500 == 0 :
        print('cost값은 : {}'.format(cost_val))
    
prediction = sess.run(H, feed_dict = {X : test_X.values, })

cost값은 : 3389934796800.0
cost값은 : nan
cost값은 : nan
cost값은 : nan
cost값은 : nan
cost값은 : nan
cost값은 : nan
cost값은 : nan
cost값은 : nan
cost값은 : nan


In [172]:
save_file(prediction, 2)
# learning_rate = 0.0000001
# 30000번 학습
# 	1857916.1853515022	 

done


In [176]:
save_file(prediction, 3)
# 300000번 학습
# 1844690.7292044302	

done


In [186]:
save_file(prediction, 5)
# num_staff','num_actor'를 추가하였을 때 
# 	1493283.9481655352	

done


In [55]:
save_file(prediction, 10)
# num_staff','num_actor'를 추가하였을 때 
# genre, screening_rat : get_dummies
# 1493679.4503123555	

done


In [ ]:
save_file(prediction, 11)
#1496128.6506115296	

In [162]:
prediction = abs(prediction.astype(int))
# 'num_staff','num_actor', 'dir_prev_num'를 추가하였을 때
# release_year 제거
# -값들을 0으로 변경

save_file(prediction, 14)

# 1492642.1951272835

done


## 3. k-fold , 

In [52]:
df.columns

Index(['dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor',
       'box_off_num', 'new_distributor', 'genre_SF', 'genre_공포', 'genre_느와르',
       'genre_다큐멘터리', 'genre_드라마', 'genre_멜로/로맨스', 'genre_뮤지컬', 'genre_미스터리',
       'genre_서스펜스', 'genre_애니메이션', 'genre_액션', 'genre_코미디', 'release_year',
       'release_month', 'release_day', 'new_time', 'genre_12세 관람가',
       'genre_15세 관람가', 'genre_전체 관람가', 'genre_청소년 관람불가'],
      dtype='object')

In [54]:
df.columns = ['dir_prev_num', 'dir_prev_num','num_staff', 'num_actor', 'box_off_num',
       'new_distributor', 'genre_SF', 'genre_horror', 'genre_noir', 'genre_document',
       'genre_drama', 'genre_romance', 'genre_musical', 'genre_mystery', 'genre_suspense',
       'genre_animation', 'genre_action', 'genre_comedy', 'release_year','release_month', 'release_day',
       'new_time', 'genre_12', 'genre_15', 'genre_all',
       'genre_19']
testdf.columns = ['dir_prev_num', 'dir_prev_num','num_staff', 'num_actor',
       'new_distributor', 'genre_SF', 'genre_horror', 'genre_noir', 'genre_document',
       'genre_drama', 'genre_romance', 'genre_musical', 'genre_mystery', 'genre_suspense',
       'genre_animation', 'genre_action', 'genre_comedy','release_year', 'release_month', 'release_day',
       'new_time', 'genre_12', 'genre_15', 'genre_all',
       'genre_19']

In [55]:
!pip install lightgbm

In [58]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
k_fold = KFold(n_splits=5, shuffle=True, random_state = 777)

model = lgb.LGBMRegressor(random_state = 777, n_estimators=1000)
features = ['dir_prev_num', 'dir_prev_num','num_staff', 'num_actor',
       'new_distributor', 'genre_SF', 'genre_horror', 'genre_noir', 'genre_document',
       'genre_drama', 'genre_romance', 'genre_musical', 'genre_mystery', 'genre_suspense',
       'genre_animation', 'genre_action', 'genre_comedy','release_year', 'release_month', 'release_day',
       'new_time', 'genre_12', 'genre_15', 'genre_all',
       'genre_19']

target = ['box_off_num']

params = {
    'learning_rate' : [0.1, 0.01, 0.003],
    'min_child_samples' : [20, 30]}

gs = GridSearchCV(estimator=model,
                 param_grid = params,
                 scoring='neg_mean_squared_error',
                 cv=k_fold)

X_train, X_test, Y_train = df[features], testdf[features], df[target]


In [59]:
gs.fit(X_train, Y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=777, shuffle=True),
             error_score=nan,
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=1000, n_jobs=-1,
                                     num_leaves=31, objective=None,
                                     random_state=777, reg_alpha=0.0,
                                     reg_lambda=0.0, silent=True, subsample=1.0,
                                     subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='deprecated', n_jobs=None,
             param_grid={'learning_rate': [0.1, 0.01, 0.003],
                    

In [60]:
gs.best_params_

{'learning_rate': 0.003, 'min_child_samples': 30}

In [62]:
model = lgb.LGBMRegressor(random_state = 777, n_estimators=1000, learning_rate= 0.003, min_child_samples= 30)
models = []
for train_idx, val_idx in k_fold.split(X_train) :
    x_t = X_train.iloc[train_idx]
    y_t = Y_train.iloc[train_idx]
    x_val = X_train.iloc[val_idx]
    y_val = Y_train.iloc[val_idx]
    
    models.append(model.fit(x_t, y_t, eval_set = (x_val, y_val), early_stopping_rounds=100, verbose=100))
    

Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 2.5656e+12
[200]	valid_0's l2: 2.47581e+12
[300]	valid_0's l2: 2.43063e+12
[400]	valid_0's l2: 2.4177e+12
[500]	valid_0's l2: 2.41058e+12
[600]	valid_0's l2: 2.3712e+12
[700]	valid_0's l2: 2.32371e+12
[800]	valid_0's l2: 2.33443e+12
Early stopping, best iteration is:
[723]	valid_0's l2: 2.31564e+12
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 5.1462e+12
[200]	valid_0's l2: 4.87113e+12
[300]	valid_0's l2: 4.65977e+12
[400]	valid_0's l2: 4.53494e+12
[500]	valid_0's l2: 4.4755e+12
[600]	valid_0's l2: 4.4531e+12
[700]	valid_0's l2: 4.41303e+12
[800]	valid_0's l2: 4.37819e+12
[900]	valid_0's l2: 4.35037e+12
[1000]	valid_0's l2: 4.34754e+12
Did not meet early stopping. Best iteration is:
[974]	valid_0's l2: 4.34403e+12
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 3.31729e+12
[200]	valid_0's l2: 3.04807e+12
[300]	valid_0's l2: 2.9388e+12
[

In [63]:
preds = []
for model in models :
    preds.append(model.predict(X_test))
len(preds)

5

In [64]:
import numpy as np
prediction = np.mean(preds, axis=0)

In [231]:
save_file(prediction, 15)

done


In [68]:
save_file(prediction, 19)
# 1479080.053011345

done


# tmp

In [ ]:
#screen_rat
def screen_rat(df) :
    tmp = sorted(df['screening_rat'].unique(), reverse=False)

    mapping = {}
    num = 1
    for i in tmp :
        mapping.update({i : num})
        num += 1

    return make_col(df, 'screening_rat', mapping)

def staff(df) :
    new_staff = []
    for i in df['num_staff'] :
        if i == 0 :
            new_staff.append(0)
        elif i <= 100 :
            new_staff.append(1)
        elif i <= 500 :
            new_staff.append(2)
        else :
            new_staff.append(3)
    df['new_staff'] = pd.DataFrame(new_staff)
    del df['num_staff']
    return df

def actor(df) :
    new_actor = []
    for i in df['num_actor'] :
        if i == 0 :
            new_actor.append(0)
        elif i <= 5 :
            new_actor.append(1)
        elif i <= 10:
            new_actor.append(2)
        else :
            new_actor.append(3)
    df['new_actor'] = pd.DataFrame(new_actor)
    del df['num_actor']
    return df


# time 
def time(df) :
    new_time = []
    for i in df['time'] :
        if i >= 90 and i < 130 :
            new_time.append(3)
        elif i > 70 : 
            new_time.append(2)
        else :
            new_time.append(1)

    df['new_time'] = pd.DataFrame(new_time)
    del df['time']
    return df

In [ ]:
n = 0
a = 0
idx = []
for i in range(len(prediction)) :
    if prediction[i] < 0 :
        prediction[i] = 0
        n += 1
        #print(i, a)
        idx.append(a)
    a += 1
#test_X.loc[idx, :].sort_values(by = 'num_staff', ascending=False)
prediction

In [167]:
for i in range(len(prediction)) :
    if prediction[i] < 0 :
        prediction[i] = 0
